# Ollama + OpenAI + Python

## 1. Tentukan nama model

Jika anda menggunakan model yang berbeza daripada "phi3:mini", ubah nilai dalam sel di bawah. Pembolehubah itu akan digunakan dalam kod sepanjang buku nota.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Sediakan klien Open AI

Biasanya, klien OpenAI digunakan dengan OpenAI.com atau Azure OpenAI untuk berinteraksi dengan model bahasa besar.  
Namun, ia juga boleh digunakan dengan Ollama, kerana Ollama menyediakan endpoint yang serasi dengan OpenAI di "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Hasilkan penyelesaian sembang

Sekarang kita boleh menggunakan SDK OpenAI untuk menghasilkan respons bagi perbualan. Permintaan ini sepatutnya menghasilkan haiku tentang kucing:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Kejuruteraan prompt

Mesej pertama yang dihantar kepada model bahasa dipanggil "mesej sistem" atau "prompt sistem", dan ia menetapkan arahan keseluruhan untuk model tersebut.  
Anda boleh menyediakan prompt sistem tersuai anda sendiri untuk membimbing model bahasa menghasilkan output dengan cara yang berbeza.  
Ubah `SYSTEM_MESSAGE` di bawah untuk menjawab seperti watak filem/TV kegemaran anda, atau dapatkan inspirasi untuk prompt sistem lain daripada [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Setelah anda menyesuaikan mesej sistem, berikan soalan pengguna pertama dalam `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Contoh beberapa tembakan

Cara lain untuk membimbing model bahasa adalah dengan memberikan "beberapa tembakan", iaitu satu siri contoh soalan/jawapan yang menunjukkan bagaimana ia sepatutnya bertindak balas.

Contoh di bawah cuba membuat model bahasa bertindak seperti pembantu pengajar dengan memberikan beberapa contoh soalan dan jawapan yang mungkin diberikan oleh seorang pembantu pengajar, dan kemudian memberikan model dengan soalan yang mungkin ditanya oleh seorang pelajar.

Cuba dahulu, dan kemudian ubah `SYSTEM_MESSAGE`, `EXAMPLES`, dan `USER_MESSAGE` untuk senario baharu.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Penjanaan Augmentasi Pengambilan

RAG (Penjanaan Augmentasi Pengambilan) adalah teknik untuk mendapatkan model bahasa menjawab soalan dengan tepat bagi domain tertentu, dengan terlebih dahulu mengambil maklumat yang relevan daripada sumber pengetahuan dan kemudian menjana respons berdasarkan maklumat tersebut.

Kami telah menyediakan fail CSV tempatan dengan data tentang kereta hibrid. Kod di bawah membaca fail CSV, mencari padanan dengan soalan pengguna, dan kemudian menjana respons berdasarkan maklumat yang ditemui. Perlu diingat bahawa ini akan mengambil masa lebih lama daripada mana-mana contoh sebelumnya, kerana ia menghantar lebih banyak data kepada model. Jika anda perasan jawapan masih tidak berasaskan data, anda boleh mencuba kejuruteraan sistem atau mencuba model lain. Secara amnya, RAG lebih berkesan dengan sama ada model yang lebih besar atau dengan versi SLM yang telah disesuaikan.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
